<a href="https://colab.research.google.com/github/Arnouux/NLP_tweets/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls "drive/MyDrive/data"
!rm -rf "twitter-datasets"
!cp -r "drive/MyDrive/data" "twitter-datasets"

embeddings
embeddings_trained.txt
embeddings_trained.txt.trainables.syn1neg.npy
embeddings_trained.txt.wv.vectors.npy
embeddings.txt
embeddings.txt.syn1neg.npy
embeddings.txt.trainables.syn1neg.npy
embeddings.txt.wv.vectors.npy
test_data_processed.txt
test_data.txt
test_shuffled_full.tsv
test_shuffled_full_zipf_luft.tsv
test_shuffled_no_processing.tsv
test_shuffled.tsv
train_shuffled_full.tsv
train_shuffled_full_zipf_luft.tsv
train_shuffled_no_processing.tsv
train_shuffled.tsv


In [ ]:
!ls "drive/MyDrive/data"

embeddings
embeddings_trained.txt
embeddings_trained.txt.trainables.syn1neg.npy
embeddings_trained.txt.wv.vectors.npy
embeddings.txt
embeddings.txt.syn1neg.npy
embeddings.txt.trainables.syn1neg.npy
embeddings.txt.wv.vectors.npy
test_data_processed.txt
test_data.txt
test_shuffled_full.tsv
test_shuffled_full_zipf_luft.tsv
test_shuffled_no_processing.tsv
test_shuffled.tsv
train_shuffled_full.tsv
train_shuffled_full_zipf_luft.tsv
train_shuffled_no_processing.tsv
train_shuffled.tsv


In [ ]:
import torch

# Preparing Data

In [ ]:
from torchtext.legacy import data
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

TEXT = data.Field(include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

train_data, test_data = data.TabularDataset.splits(
                                        path = 'twitter-datasets',
                                        train = 'train_shuffled.tsv',
                                        test = 'test_shuffled.tsv',
                                        format = 'tsv',
                                        fields = [("text", TEXT), ("label", LABEL)],
)

MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.twitter.27B.200d", 
                 unk_init = torch.Tensor.normal_)
LABEL.build_vocab(train_data)

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

.vector_cache/glove.twitter.27B.zip: 1.52GB [04:49, 5.25MB/s]                            
100%|█████████▉| 1193513/1193514 [01:48<00:00, 11034.27it/s]


In [ ]:
print(vars(train_data.examples[0]))

{'text': ['<user>', 'waaay', 'back', 'when', '...', 'i', 'miss', 'it'], 'label': '0'}


# Using W2V

In [ ]:
import multiprocessing
cores = multiprocessing.cpu_count()
print(cores)

2


In [ ]:
sentences = []
with open('twitter-datasets/train_shuffled_full.tsv', 'r') as f :
  line = f.readline()
  while line != "":
    sentences.append(line[:-3].split())
    line = f.readline()
  
with open('twitter-datasets/test_shuffled_full.tsv', 'r') as f :
  line = f.readline()
  while line != "":
    sentences.append(line[:-3].split())
    line = f.readline()

In [ ]:
# adding test data sentences

with open('twitter-datasets/test_data.txt', 'r') as f :
  line = f.readline()
  while line != "":
    line_arr = line.split(",")
    line = ""
    for l in line_arr[1:]:
      line += l
    sentences.append(line)
    line = f.readline()

In [ ]:
len(sentences)
sentences[0]

['i', 'wish', 'i', 'lived', 'in', 'england', '0']

In [ ]:
import gensim

In [ ]:
from gensim.models import Word2Vec
w2v_model = Word2Vec(sentences, size=100, min_count=1, workers=cores-1, sg=0)

In [ ]:
# Train Word Embeddings
w2v_model.train(sentences, total_examples=len(sentences), epochs=100)

In [ ]:
w2v_model.save('embeddings.txt')

In [ ]:
print(gensim.__version__)

3.6.0


In [ ]:
import gensim
w2v_model = gensim.models.Word2Vec.load('twitter-datasets/embeddings_trained.txt') # change for embeddings_trained # trained using different python version (3.8) leads to error
weights = torch.FloatTensor(w2v_model.wv.vectors)
embedding = torch.nn.Embedding.from_pretrained(weights)

# Preprocessing data

In [ ]:
import re
from nltk.metrics.distance import jaccard_distance
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, words
from nltk.stem import PorterStemmer
from nltk.util import ngrams
import random

import nltk
nltk.download('stopwords')
nltk.download('punkt')

correct_words = words.words()
stop_words = stopwords.words()
ps = PorterStemmer()

def clean_sentence(val):
    """remove chars that are not letters or numbers, downcase, 
    then remove stop words"""
    regex = re.compile('([^\s\w]|_)+')
    sentence = regex.sub('', val).lower()
    sentence = word_tokenize(sentence)
    for word in list(sentence):
        if word in stop_words:
            sentence.remove(word)
        else:
            words_list = [(jaccard_distance(set(ngrams(word, 2)), set(ngrams(w, 2))),w) for w in correct_words if w[0]==word[0]]
            try:
                word_best = sorted(words_list, key = lambda val:val[0])[0][1]
            except:
                word_best = word
            sentence[sentence.index(word)] = ps.stem(word_best)
    sentence = " ".join(sentence)
    return sentence

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
with open('twitter-datasets/train_shuffled.tsv', 'r') as f :
  with open('twitter-datasets/train_shuffled_processed.tsv', 'w') as f2 :
    line = f.readline()
    while line != "" :
      line = f.readline() # [:-3] ~ removes /t <int> /t
      print(line[:-3])
      f2.write(clean_sentence(line[:-3]) + line[-3:] )

#9221 we have had good times in the past & i miss my lil pothead
sanyo rl - 4920 directsync - mobile charging kit ( high current output ) ( uk outlet plug the directsync - mobile c ... <url>
<user> i thought i answered on fb that i work :p sorrry ! ! ! ya i cant today ( (


ZeroDivisionError: ignored

# Model

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        # lengths need to be on CPU!
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)

In [ ]:
INPUT_DIM = 25000
EMBEDDING_DIM = 200
HIDDEN_DIM = 128
OUTPUT_DIM = 1
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            )

In [ ]:
print(model)

RNN(
  (embedding): Embedding(25000, 200)
  (rnn): LSTM(200, 128, num_layers=3, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=256, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.9410, -1.0278, -0.2021,  ...,  1.0364,  0.0901, -0.1600],
        ...,
        [ 0.0159, -0.8099, -0.1014,  ..., -0.1012,  0.2796, -0.7594],
        [-0.8072,  0.5999,  1.3365,  ..., -0.8323, -2.0134,  0.6976],
        [ 0.3274, -0.7810, -1.2018,  ..., -0.8354,  0.2926, -2.4191]])


In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 6,129,105 trainable parameters


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
device

device(type='cuda')

In [ ]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# Training

In [ ]:
N_EPOCHS = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 1m 25s
	Train Loss: 0.359 | Train Acc: 83.56%


# Predictions

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [ ]:
tokenized = {'text': [tok.text for tok in nlp.tokenizer("  ... and he loses 2 strength points")]}
print(vars(train_data.examples[0]))
tokenized

{'text': ['<user>', 'waaay', 'back', 'when', '...', 'i', 'miss', 'it'], 'label': '0'}


{'text': ['  ', '...', 'and', 'he', 'loses', '2', 'strength', 'points']}

In [ ]:
line = "This film is terrible".split(",")

predict_sentiment(model, "it is working good")

0.04103153198957443

In [ ]:
x = 0
predictions = []
ids = []
with open('twitter-datasets/test_data_processed.txt', 'r') as f :
  line = f.readline()
  while line != "" :
    line_arr = line.split(",")
    line = ""
    for l in line_arr[1:]:
      line += l
    y_pred = predict_sentiment(model, line)
    if y_pred > 0.5:
      y_pred = -1
    else:
      y_pred = 1
    predictions.append(y_pred)
    ids.append(line_arr[0])
    line = f.readline()
    x += 1
    if x % 1000 == 0:
      print(f"sentence {x}")

sentence 1000
sentence 2000
sentence 3000
sentence 4000
sentence 5000
sentence 6000
sentence 7000
sentence 8000
sentence 9000
sentence 10000


In [ ]:
import csv
def create_csv_submission(ids, y_pred, name):
    """
    Creates an output file in .csv format for submission to Kaggle or AIcrowd
    Arguments: ids (event ids associated with each prediction)
               y_pred (predicted class labels)
               name (string name of .csv output file to be created)
    """
    with open(name, 'w') as csvfile:
        fieldnames = ['Id', 'Prediction']
        writer = csv.DictWriter(csvfile, delimiter=",", fieldnames=fieldnames)
        writer.writeheader()
        for r1, r2 in zip(ids, y_pred):
            writer.writerow({'Id':int(r1),'Prediction':int(r2)})

create_csv_submission(ids, predictions, "submission.csv")

In [ ]:
!ls

drive  sample_data  submission.csv  twitter-datasets


In [ ]:
from google.colab import files
files.download('submission.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>